In [ ]:
try:
  #mount google drive
  from google.colab import drive
  drive_path="/content/drive"
  drive.mount(drive_path)
  drive_folder=drive_path+"/MyDrive/dtm/"
  using_colab=True
except ModuleNotFoundError:
  # Assume we are not on google colab, 
  drive_folder = "data/"
  using_colab=False
  pass

# Params and Init



In [ ]:
print("Importing    ")
!pip -q install --upgrade pip
# !pip -q install --upgrade tensorflow keras
!pip -q install plyfile deepdiff talos GitPython numba
!pip -q install --upgrade imgaug
import talos
from deepdiff import DeepDiff
import re
import numpy as np
from progressbar import progressbar, ProgressBar
import sys
import os
import cv2
import math
import json
import importlib
import matplotlib.pyplot as plt
import tensorflow as tf
import git
from tensorflow import keras
import keras.backend as K
import datetime
import imgaug.augmenters as iaa
cuda_paths=!ls /usr/local | grep cuda-
os.environ['CUDA_HOME'] = os.path.join("/usr/local/",cuda_paths[-1])
K.manual_variable_initialization(True)

In [ ]:
print("Importing deep_ga")
if using_colab:
    !git -C /content/deep_ga/ pull || git clone https://github.com/InigoMoreno/deep_ga
    sys.path.append('/content/deep_ga')
    import deep_ga  # nopep8
    deep_ga=importlib.reload(deep_ga)
else:
    MODULE_PATH = "deep_ga/deep_ga/__init__.py"
    MODULE_NAME = "deep_ga"
    git.cmd.Git(MODULE_NAME).pull()
    spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
    deep_ga = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = deep_ga
    spec.loader.exec_module(deep_ga)


In [ ]:
p={
  "resolution" : .5,                     # resolution of map [meters per pixel]
  "mapLength"  : 40,                     # size of one side of the map [meters]
  "minSlopeThreshold"  : 0.5,            # minimum slope to be counted [proportion]
  "maxNanPercentage"   : 5/100,          # maximum percentage of NaNs in a patch [%]
  "minSlopePercentage" : 0/100,          # minimum percentage of slope in a patch [%]
  "maxSlopePercentage" : 100/100,        # maximum percentage of slope in a patch [%]
  "stdPatchShift"          : 15,         # standard deviation of shift between to patches [m]
  "local_resolution" : .5,
  "local_mapLength"  : 40,
  "local_maxNanPercentage": 65/100,
  "local_global_minCcorr": 15/100,
   "booleanDist": False
   
}
p["mapLengthPixels"]=math.ceil(p["mapLength"]/p["resolution"])
p["local_mapLengthPixels"]=math.ceil(p["local_mapLength"]/p["local_resolution"])
deep_ga.set_scale(p["stdPatchShift"])

p["augment_a"] = iaa.Sequential([
    iaa.Add((-4, 4)),
    iaa.PerspectiveTransform(scale=0.01),
    iaa.BlendAlphaSimplexNoise(iaa.Add(0.2), upscale_method="cubic", size_px_max=4),
])


hyperparams = {
    # "input": ["raw", "rawmask", "sobel", "fixsobel", "PConv", "SymConv", "Conv"],
    "input": ["SymConv"],
    # "batch_size": [32, 64, 128, 256],
    "batch_size": [4],
    # "mobileNet_alpha": [1.0, 0.5, 0.35],
    "mobileNet_alpha": [1.0],
    # "mobileNet_pooling": ["avg", "max", None],
    "mobileNet_pooling": ["avg"],
    "mobileNet_weights": [None],
    # "mobileNet_weights": ["imagenet"],
    # "firstLayerSize": [4096, 1280, 1000],"
    "firstLayerSize": [0],
    # "dropout": [0, 0.2, 0.5],
    "dropout": [0.01],
    # "secondLayerSize": [0, 100, 25],
    "secondLayerSize": [50],
    # "activation": ["relu", None],
    "activation": [None],
    # "sharedWeights": [True, False],
    "sharedWeights": [True],
    # "loss": ["MSE", "MAE", "DOOMSE", "PCL", "BCE"],
    "loss": ["DOOMSE"],
    "optimizer": ["SGD"],
    # "learning_rate": [0.01, 0.001, 0.0001],
    "learning_rate": [0.0001],
    # "learnEnding": [True],
    "learnEnding": [False],
    # "loadFolder": [os.path.join(drive_folder,
    #                             "talos_oxia_mobileNet_alpha__secondLayerSize")]
}
old_hyperparam_tests = [
    {"input": ["SymConv", "raw", "rawmask", "sobel", "fixsobel", "PConv", "Conv"],
    "repeat": [1,2,3]},#0
    {"displace": [True, False],
    "repeat": [1,2,3]}, #1
    {"batch_size": [2, 4, 8, 16, 32]}, #2
    {"loss": ["MSE", "MAE", "DOOMSE", "PCL"]}, #3
    {"optimizer": ["Adam", "SGD"],
     "learning_rate": [0.01, 0.001, 0.0001]}, #4
    {"mobileNet_alpha": [1.4, 1.0, 0.75, 0.5, 0.35],
    "repeat": [1,2,3]}, #5
    {"mobileNet_pooling": ["avg", "max"],
    "repeat": [1,2,3]}, #6
    {"mobileNet_weights": [None,"imagenet"]}, #7
    {"firstLayerSize": [500,100,50,25,20,15,10]}, #8
    {"dropout": [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
     "secondLayerSize": [100]}, #9
    {"activation": [None, "relu"],
     "secondLayerSize": [100],
     "repeat": [1,2]}, #10
    {"sharedWeights": [True, False],
    "repeat": [1,2,3]}, #11
]
hyperparam_tests = [
    {"sharedWeights": [True, False],
    "repeat": [1,2,3]}, #0
    {"mobileNet_pooling": ["avg", "max"],
    "repeat": [1,2,3]}, #1
    {"input": ["SymConv", "raw", "rawmask", "sobel", "fixsobel", "PConv", "Conv"],
    "arepeat": [1,2,3]},#2
    {"displace": [True, False],
    "arepeat": [1,2,3]}, #3
    {"mobileNet_alpha": [1.4, 1.0, 0.75, 0.5, 0.35],
    "arepeat": [1,2,3]}, #4
    {"dropout": [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
     "aarepeat": [1,2]}, #5
    {"mobileNet_weights": [None,"imagenet"],
     "arepeat": [1,2,3]}, #6
]


if len(sys.argv)==2:
    try: 
        idx=int(sys.argv[1])
        hyperparams = {**hyperparams, **hyperparam_tests[idx]}
    except ValueError:
        pass

keys= [k for k,v in hyperparams.items() if len(v)>1]
experiment_name="talos_tenerife_"+"__".join(keys)
experiment_path = os.path.relpath(os.path.join(drive_folder,experiment_name))
print(experiment_name)
first_hyperparams={k:v[0] for k,v in hyperparams.items()}

# Dataset preparation

In [ ]:
#get local_dems
with np.load(os.path.join(drive_folder,"processed","filtered.npz")) as data:
  dems=data["dems"]
  gps= data["gps"]
#get global_dem
ply_file=os.path.join(drive_folder,"minas_densified_point_cloud.ply")
global_dem , global_img, displacement = deep_ga.ply_to_image(ply_file,p["resolution"])

In [ ]:
day = np.vectorize(lambda a: datetime.datetime.utcfromtimestamp(a).day)(gps[:,0]/1e6)
validation = day==8

tdems=dems[~validation,:,:]
vdems=dems[validation,:,:]
tgps=gps[~validation,:]
vgps=gps[validation,:]

Nv=vdems.shape[0]
Nt=tdems.shape[0]
print(f"Validation split: {Nv/(Nt+Nv)*100:.1f}%")

# del dems, gps

In [ ]:
patches_a, patches_b,distances=deep_ga.get_batch_local_global(1,dems,gps,global_dem,displacement,p,seed=806)


for i in range(1):
  W=p["local_mapLength"]/2
  print(distances[i])
  # ax=plt.subplot(1,2,1)

  # plt.title("Local DEM")
  # plt.fig
  # plt.xlabel("x [m]")
  # plt.ylabel("y [m]")
  plt.imshow(patches_a[i,:,:],extent=(-W,W,-W,W))
  plt.savefig(f"tenerife.pdf",  dpi=500, bbox_inches = 'tight',
    pad_inches = 0)
  plt.show()

  # W=p["mapLength"]/2
  # plt.subplot(1,2,2)
  # plt.title("Global DEM")
  # plt.xlabel("x [m]")
  # # plt.ylabel("y [m]")
  plt.imshow(patches_b[i,:,:],extent=(-W,W,-W,W))
  # # plt.savefig(f"tenerife{seed}.pdf",  dpi=500, bbox_inches = 'tight',
  # #   pad_inches = 0)
  plt.show()
# plt.hist(distances)
# plt.show()

# Model preparation

In [ ]:
if using_colab:
    %load_ext tensorboard
    %tensorboard --logdir {os.path.join(experiment_path,"logs")}

In [ ]:
import datetime

def talos_model(x_train, y_train, x_val, y_val, params):
  global experiment_path, keys

  values=[str(params[k]) for k in keys]
  i="__".join(values)
  i=i.replace(".","_")
  print(i)


  if "displace" in params.keys() and hyperparams["displace"]:
    dx,dy,dz = displacement
    dx2=4
    dy2=3
    displace=(dx+dx2,dy+dy2,dz)
  else:
    displace=displacement

  # get data generator
  tgenerator=deep_ga.LocalGlobalPatchDataGenerator(tdems.shape[0]/5,params["batch_size"],tdems,tgps,global_dem,displace,p)

  # get validation data generator
  vp=p.copy()
  vp["augment_a"]=None
  vgenerator=deep_ga.LocalGlobalPatchDataGenerator(vdems.shape[0],params["batch_size"],vdems,vgps,global_dem,displace,vp)
  
  with open(os.path.join(experiment_path, f"{i}_params.json"), 'w') as fp:
      json.dump(params, fp)

  callbacks=[]

  # stop early if loss does not improve
  callbacks.append(keras.callbacks.EarlyStopping(
      monitor='val_loss', patience=20, verbose=1, restore_best_weights=True,min_delta=0.1))
  
  # assert that weights are finite
  callbacks.append(deep_ga.AssertWeightsFinite())

  # plot prediction
  callbacks.append(deep_ga.PlotPrediction(
      folder=os.path.join(experiment_path, f"{i}_plots"),
      tgen=tgenerator,
      vgen=vgenerator,
      save=not using_colab
    ))
  
    
  # tensorboard callback
  logdir = os.path.join(experiment_path,"logs", f"{i}")
  callbacks.append(tf.keras.callbacks.TensorBoard(logdir))

  # get model
  input_a = keras.Input(shape=(
      p["local_mapLengthPixels"], p["local_mapLengthPixels"]), name="patch_a")
  input_b = keras.Input(
      shape=(p["mapLengthPixels"], p["mapLengthPixels"]), name="patch_b")

  model = deep_ga.get_model(params,input_a,input_b)
  model = deep_ga.compile_model(model, y_val, params)

  out = model.fit(tgenerator, 
                  validation_data=vgenerator,
                  epochs=200, 
                  workers=6,
                  callbacks=callbacks,
                #   use_multiprocessing = True,
                  verbose=(1 if using_colab else 2)
                  )
                  
  x,y=tgenerator.get_batch(320)
  yp=model.predict(x)[:,0]
  print(f"self mse loss = {keras.losses.MSE(y,yp)}")
  try:
    with open(os.path.join(experiment_path, f"{i}_structure.json"), 'w') as fp:
        fp.write(model.to_json())
    model.save_weights(os.path.join(experiment_path, f"{i}_weights.h5"))
    model.save(os.path.join(experiment_path, f"{i}"),include_optimizer=False)
    model.save(os.path.join(experiment_path, f"{i}.h5"))
  except:
    pass

  return out, model


In [ ]:
fake_local=np.empty((100,p["local_mapLengthPixels"],p["local_mapLengthPixels"]),np.float32)
fake_global=np.empty((100,p["mapLengthPixels"],p["mapLengthPixels"]),np.float32)
fake_out = np.empty(100)

i=0
import shutil
if os.path.exists(experiment_path):
  shutil.rmtree(experiment_path)

scan_object = talos.Scan(
    x=[fake_local, fake_global], y=fake_out, 
    params=hyperparams, 
    model=talos_model,
    experiment_name = experiment_path,
    time_limit = (datetime.datetime.now() + datetime.timedelta(hours=40)).strftime("%Y-%m-%d %H:%M"),
    minimize_loss=True,
    save_weights=True,
    print_params=True,
    random_method='quantum',
    val_split=0
  )


In [ ]:
import pickle


def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, protocol=2)


def project_object(obj, *attributes):
    out = {}
    for a in attributes:
        out[a] = getattr(obj, a)
    return out


pickle_path = os.path.join(experiment_path, "scan_object.pickle")
t = project_object(scan_object, 'params', 'saved_models',
                   'saved_weights', 'data', 'details', 'round_history')
save_object(t, pickle_path)



Copy this on the console to prevent termination:
```
function ClickConnect() {
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect, 60000)
```

Interesting resources: 
 - Hyperparameter optimization with talos
 - Tinyfying Neural Network with uTensor or TinyML

